In [ ]:
# !pip install scikit-learn==1.1.0 #diffprivlib contractions
!pip install optuna
!pip install numpy==1.24.4  # Stable with torch 2.x and opacus
!pip install --upgrade opacus

import numpy as np
import matplotlib.pyplot as plt
from opacus import PrivacyEngine

from tqdm.notebook import tqdm
from scipy.stats import gamma
import os as os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import time

import matplotlib.pyplot as plt
import math
import torch.optim as optim

# For confidence intervals
import scipy
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Utility Functions

In [ ]:
# Chebyshev approximation polynomial
def phi_logit(x):
    return -math.log(1 + math.exp(-x))

class Chebyshev:
    """
    Chebyshev(a, b, n, func)
    Given a function func, lower and upper limits of the interval [a,b],
    and maximum degree n, this class computes a Chebyshev approximation
    of the function.
    Method eval(x) yields the approximated function value.
    """

    def __init__(self, a, b, n, func):
        self.a = a
        self.b = b
        self.func = func
        self.n = n
        bma = 0.5 * (b - a)
        bpa = 0.5 * (b + a)
        f = [func(math.cos(math.pi * (k + 0.5) / n) * bma + bpa) for k in range(n)]
        fac = 2.0 / n
        self.c = [fac * sum([f[k] * math.cos(math.pi * j * (k + 0.5) / n)
                             for k in range(n)]) for j in range(n)]

    def eval(self, x):
        a, b = self.a, self.b
        y = (2.0 * x - a - b) * (1.0 / (b - a))
        y2 = 2.0 * y
        (d, dd) = (self.c[-1], 0)
        for cj in self.c[-2:0:-1]:
            (d, dd) = (y2 * d - dd + cj, d)
        return y * d - dd + 0.5 * self.c[0]

    def monomial_coeffs(self):
        """
        Converts the Chebyshev approximation (assumed n >= 3) into a
        standard polynomial form: f(x) ≈ a0 + a1*x + a2*x^2
        """
        if self.n < 3:
            raise ValueError("monomial_coeffs() requires at least a second-order (n >= 3) approximation.")

        c0, c1, c2 = self.c[0], self.c[1], self.c[2]
        a, b = self.a, self.b
        alpha = 2 / (b - a)
        beta = -(a + b) / (b - a)

        a0 = c0 - c2 + c1 * beta + 2 * c2 * beta**2
        a1 = c1 * alpha + 4 * c2 * alpha * beta
        a2 = 2 * c2 * alpha**2

        return [a0, a1, a2]

# ----------------------------------------------------------
# 2. Simple CNN
# ----------------------------------------------------------
class SmallCNN(nn.Module):
    def __init__(self, output_dim = 128, num_classes = 2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 128), nn.ReLU(),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

def extract_cnn_features(model, loader, device=torch.device('cpu')):
    """
    Returns:
        feats  : torch.Tensor  [N, 128]
        labels : torch.Tensor  [N]
    """
    model.eval()                       # inference mode
    feats, labels = [], []

    with torch.no_grad():
        for x, y in tqdm(loader, desc="Extracting"):
            x = x.to(device)
            # --- forward until the 128‑dim layer ---
            h = model.features(x)                # shape: [b, 64, 8, 8]
            h = h.view(h.size(0), -1)            # flatten
            h = model.classifier[0](h)           # Linear(64*8*8 → 128)
            h = model.classifier[1](h)           # ReLU

            feats.append(h.cpu())
            labels.append(y)

    feats  = torch.cat(feats)   # [N, 128]
    labels = torch.cat(labels)  # [N]
    return feats, labels

##################### Load Datasets #####################
def filter_and_relabel(dataset, digits):
    idx = [i for i, y in enumerate(dataset.targets) if int(y) in digits]
    imgs = torch.stack([dataset[i][0] for i in idx])
    labels = torch.tensor([0 if dataset.targets[i] == digits[0] else 1 for i in idx])
    return TensorDataset(imgs, labels)

def load_CIFAR10(digits, batch_size):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (1.0,))
    ])

    train_ds = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_ds  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    train_set = filter_and_relabel(train_ds, digits)
    test_set  = filter_and_relabel(test_ds,  digits)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader  = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

def load_CIFAR100(digits, batch_size):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (1.0,))
    ])

    train_ds = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
    test_ds  = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

    train_set = filter_and_relabel(train_ds, digits)
    test_set  = filter_and_relabel(test_ds,  digits)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader  = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

def load_MNIST(digits, batch_size):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.Grayscale(num_output_channels=3),  # no-op for RGB images
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (1.0,))
    ])

    train_ds = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_ds  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    train_set = filter_and_relabel(train_ds, digits)
    test_set  = filter_and_relabel(test_ds,  digits)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader  = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

def load_FMNIST(digits, batch_size):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.Grayscale(num_output_channels=3),  # no-op for RGB images
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (1.0,))
    ])

    train_ds = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
    test_ds  = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

    train_set = filter_and_relabel(train_ds, digits)
    test_set  = filter_and_relabel(test_ds,  digits)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader  = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

##################### Logistic Regression Utilities #####################

# Compute the derivatives of the logistic loss function
def logistic_derivatives(X, y, theta, n=1):
    # Compute θ^T x_i for each i
    t = X @ theta  # shape (n,)
    # σ(θ^T x_i)
    p = 1.0 / (1.0 + np.exp(-t))
    # d_i = p - y_i
    d = p - y
    return d/n

def sample_from_density(d, c):
    """
    Samples a random vector b from the density h(b) ∝ exp(-c * ||b||).

    Parameters:
    - d: Dimension of the vector b
    - c: Parameter controlling the density

    Returns:
    - b: A sampled vector from the desired density
    """
    # Step 1: Sample the norm of b from Gamma(d, 1/c)
    norm_b = np.random.gamma(shape=d, scale=1/c)

    # Step 2: Sample a random direction uniformly on the unit sphere
    direction = np.random.normal(size=d)
    direction /= np.linalg.norm(direction)

    # Step 3: Combine norm and direction
    b = norm_b * direction

    return b

# calculate values of \sigma_1 and \sigma_2 for (\alpha,\eps)-Renyi-DP
def objective_func(alpha, k, d, sigma1, sigma2, delta,C_max):
    if alpha <= 1 or alpha >= np.min((sigma1,sigma2))/(1+C_max):
        return np.inf  # Penalize out-of-bound values

    term1 = (k * alpha) / (2 * (alpha - 1)) * np.log(1.0 - (1.0 + C_max) / (np.max((sigma1, sigma2))))
    term2 = - (k / (2 * (alpha - 1))) * np.log(1 - (alpha*(1 + C_max)) / np.min((sigma1,sigma2)))
    term3 = (np.log(3.0 / delta) + (alpha - 1)*np.log(1-1/alpha) - np.log(alpha)) / (alpha - 1)
    term4 = np.sqrt(2.0 * np.log(3.75/delta)) / (sigma1/np.sqrt(k))

    return term1 + term2 + term3 + term4

def solve_sigma_renyi(sigma_DP, n_prime, d, delta, target_epsilon, C_max):
    # Define binary search bounds
    left, right = sigma_DP / 30000.0, 30000.0*sigma_DP
    best_sigma = right  # Default to upper bound in case no solution is found
    while right - left > 1e-6:  # Precision threshold
        mid_sigma = (left + right) / 2
        # Solve for optimal alpha given the current sigma
        result = scipy.optimize.minimize_scalar(objective_func,
                                 bounds=(1 + 1e-5, mid_sigma - 1e-5),
                                 args=(n_prime, d, mid_sigma, mid_sigma, delta,C_max),
                                 method='bounded')
        if result.success and result.fun < target_epsilon:
            best_sigma = mid_sigma  # Update best found sigma
            right = mid_sigma  # Search for a smaller sigma
        else:
            left = mid_sigma  # Increase sigma to meet target_epsilon
    return best_sigma

## Logistic Regression DP-Training Procedure

In [ ]:
# ---------- helper : logistic loss with ℓ2‑regularisation ----------
def lr_loss(w, X, y, lam):
    """Binary‑logistic loss + (lam/2)||w||²  (expects y∈{0,1})."""
    logits = X @ w
    data_loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, y, reduction='mean')
    reg_loss  = 0.5 * lam * w.pow(2).sum()
    return data_loss + reg_loss

# ---------- Objective‑Perturbation with L‑BFGS ----------
def Objective_perturb_LBFGS(X_train, y_train, X_test, y_test,\
                            lambda_param, epsilon, delta,\
                            num_steps=500, tol=1e-6, verbose=False,\
                            device=torch.device('cpu')):

    # --- move data to torch tensors ---
    Xtr = torch.tensor(X_train, dtype=torch.float32, device=DEVICE)
    ytr = torch.tensor(y_train, dtype=torch.float32, device=DEVICE)
    Xte = torch.tensor(X_test,  dtype=torch.float32, device=DEVICE)
    yte = torch.tensor(y_test,  dtype=torch.float32, device=DEVICE)

    n, d = Xtr.shape

    # --- sample perturbation vector b ~ N(0, I_d) ---
    b_noise     = torch.randn(d, device=DEVICE)
    sigma_scale = np.sqrt((4.0*epsilon + 8.0*np.log(2.0/delta)) / (epsilon**2))
    b_vec       = sigma_scale * b_noise
    Delta       = 1.0/(2.0 * epsilon)

    # --- initialise parameter vector ---
    w = torch.zeros(d, device=DEVICE, requires_grad=True)

    # --- L‑BFGS solver ---
    optimizer = optim.LBFGS([w], max_iter=num_steps, tolerance_grad=tol, tolerance_change=1e-20)

    # closure with perturbation term  (b·w)/n
    def closure():
        optimizer.zero_grad()
        loss = lr_loss(w, Xtr, ytr, lambda_param) + b_vec.dot(w) / n + w.dot(w) * Delta/(4.0 * n)
        loss.backward()
        return loss

    t0 = time.time()
    optimizer.step(closure)
    t1 = time.time()

    # --- evaluate on test set ---
    with torch.no_grad():
        probs  = torch.sigmoid(Xte @ w)
        preds  = (probs >= 0.5).long()
        acc    = (preds == yte.long()).float().mean().item()

    return acc, t1 - t0

# Solve with chebyshev approximation
def second_order_cheb(X_train, X_test, y_train, y_test, k, n, d, tau, sigma_matrix, Q = 6):

  cheb = Chebyshev(-Q, Q, 3, phi_logit)
  c1 = cheb.c[1]
  c2 = cheb.c[2]

  # Build noisy dataset
  N = np.random.randn(k, d)
  N_y = np.random.randn(k)

  # Calculate lambda_min
  y_col = y_train.reshape(-1, 1) if y_train.ndim == 1 else y_train
  XY = np.hstack((X_train, y_col))
  lambda_min = np.real(np.min(np.linalg.eigvals(XY.T @ XY)))
  sigma_eigenval = sigma_matrix/np.sqrt(k)

  # Compute projected data
  time_start = time.time()
  S = np.random.randn(k, n)
  if sigma_matrix <= tau:
      X_PR = S @ X_train + np.sqrt(sigma_matrix) * N  # (n_prime,d)
      y_PR = (S @ (2.0 * y_train.reshape(-1, 1) - 1.0)).ravel() + np.sqrt(sigma_matrix) * N_y
  else:
      gamma_tilde = np.max((0, lambda_min - np.sqrt(sigma_eigenval) * (tau - np.random.randn())))
      sigma_tilde = np.sqrt(np.max((0, sigma_matrix - gamma_tilde)))
      X_PR = S @ X_train + sigma_tilde * N  # (n_prime,d)
      y_PR = (S @ (2.0 * y_train.reshape(-1, 1) - 1.0)).ravel() + sigma_tilde * N_y

  # Solve over the noised sufficient statistics
  theta_acc = -(c1/(2.0 * c2)) * np.linalg.inv(X_PR.T @ X_PR) @ (X_PR.T @ y_PR)
  time_end = time.time()
  total_time = time_end - time_start

  # Calculate accuracy
  t_test = X_test @ theta_acc
  p_test = 1.0 / (1.0 + np.exp(-t_test))
  preds = (p_test >= 0.5).astype(int)

  # return final accuracy and time
  return np.mean(preds == y_test), total_time, theta_acc

### Training Functions

In [ ]:
# Train with DP-SGD
def train_with_dp(model, optimizer, train_loader, test_loader, epochs, lr, max_grad_norm,
                    delta, eps, device="cpu"):
    """
    Trains the given model with DP-SGD using Opacus.
    Returns:
        final_epsilon, test_error, training_time
    """

    model.train()
    # Move model to device
    model = model.to(device)

    # Attach privacy engine
    privacy_engine = PrivacyEngine()
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        target_epsilon=eps,
        target_delta=delta,
        epochs=epochs,
        max_grad_norm=max_grad_norm
    )

    criterion = nn.CrossEntropyLoss()

    start_time = time.time()
    for epoch in tqdm(range(epochs)):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    training_time = time.time() - start_time

    # Compute final epsilon
    epsilon = privacy_engine.get_epsilon(delta=delta)

    # Evaluate model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100.0 * correct / total
    test_error = 100.0 - test_accuracy

    return epsilon, test_accuracy, training_time, model


# Load Dataset

In [ ]:
# Choose dataset from the next options
# dataset_type = 'CIFAR10'
# dataset_type = 'CIFAR10'
# dataset_type = 'MNIST'
# dataset_type = 'FMNIST'
dataset_type = 'MNIST'

digits = [3, 8]
batch_size = 500

if dataset_type == 'CIFAR10':
    train_loader, test_loader = load_CIFAR10(digits, batch_size)
elif dataset_type == 'CIFAR100':
    digits = [3, 8]
    batch_size = 500
    train_loader, test_loader = load_CIFAR100(digits, batch_size)
elif dataset_type == 'MNIST':
    digits = [3, 8]
    batch_size = 500
    train_loader, test_loader = load_MNIST(digits, batch_size)
elif dataset_type == 'FMNIST':
    digits = [3, 8]
    batch_size = 500
    train_loader, test_loader = load_FMNIST(digits, batch_size)

# Baseline Model (Non-Private)

In [ ]:
lr_full = 0.001
epochs = 20
output_dim = 128

model_non_private = SmallCNN(output_dim=output_dim, num_classes=2)
optimizer = optim.Adam(model_non_private.parameters(), lr=lr_full, weight_decay=0.0)
criterion = nn.CrossEntropyLoss()

# Baseline Training
model_non_private = model_non_private.to(DEVICE)

# Training loop
model_non_private.train()
start_time = time.time()
for epoch in tqdm(range(epochs)):
    total_loss = 0.0
    num_batches = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model_non_private(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Accumulate loss
        total_loss += loss.item()
        num_batches += 1

    # Print the average loss for the epoch
    avg_loss = total_loss / num_batches
    print(f"Epoch [{epoch+1}/{epochs}] - Average Train Loss: {avg_loss:.4f}")
training_time = time.time() - start_time

# Evaluate model on the test set
model_non_private.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model_non_private(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
baseline_test_acc = correct / total
print('Baseline (Non-Private) Test accuracy: ' + str(100.0 * baseline_test_acc))


# Set hyperparameters

In [ ]:
# Privacy parameters
epsilon_values = np.logspace(-1.0, 1.5, 5)
delta_DP = np.min((1e-6, 0.01/len(train_loader.dataset)))
clip_norm_dpsgd = 4.0
removal_size = 10

# Number of Monte-Carlo iteraitons
iters = 250

# Confidence Intervals
confidence = 0.95  # Change to your desired confidence level
t_value = scipy.stats.t.ppf((1 + confidence) / 2.0, df=iters - 1)

# Calculate the variance of the noise for ensuring privacy of our algorithm
delta_DP_adjusted       = ((removal_size-1)/removal_size) * delta_DP
tau = np.sqrt(2.0 * np.log(3.0/delta_DP_adjusted))

## Use optimal hyperparameters

In [ ]:
if dataset_type == 'MNIST':
  k_opt_second_order = int(4.0 * output_dim)

elif dataset_type == 'CIFAR10':
  k_opt_second_order = int(5.0 * output_dim)

elif dataset_type == 'CIFAR100':
  k_opt_second_order = int(6.5 * output_dim)

elif dataset_type == 'FMNIST':
  k_opt_second_order = int(4.0 * output_dim)

### Baseline train with DP

In [ ]:
model_hyperparameters = SmallCNN(output_dim=output_dim, num_classes=2)
optimizer = optim.Adam(model_hyperparameters.parameters(), lr=lr_full, weight_decay=0.0)
_, _, _, model_dp = train_with_dp(model_hyperparameters, optimizer, train_loader, test_loader, epochs, lr_full, clip_norm_dpsgd,
                    delta_DP/removal_size, epsilon_values[-1]/removal_size, device=DEVICE)
# Extract Features
train_feats, train_labels = extract_cnn_features(model_dp, train_loader, DEVICE)
test_feats,  test_labels  = extract_cnn_features(model_dp, test_loader,  DEVICE)
# Convert to NumPy for scikit‑learn
X_train = train_feats.numpy()
y_train = train_labels.numpy()
X_test  = test_feats.numpy()
y_test  = test_labels.numpy()
norm_fact = np.sqrt(np.max(np.sum(X_train**2, 1)))
X_train = X_train/norm_fact
X_test = X_test/norm_fact
n, d = X_train.shape

# Main For Loop: Simulate optimal hyperparameter setting

In [ ]:
opt_acc_ours = []
opt_acc_ours_std = []

opt_acc_objective = []
opt_acc_objective_std = []

total_time_ours = 0.0
total_time_obj_perturb = 0.0
n, d = X_train.shape
lambda_param = 0.0
###############################################################################
# Outer loops: iterate over learning_rate_list and iters_GD_list
###############################################################################

# Loop over different epsilon values
for eps_idx, eps in tqdm(enumerate(epsilon_values)):
    print('Started eps = ' + str(eps))

    curr_test_acc = []
    curr_test_acc_loss_perturb = []

    # Zero the time counters only for n_prime == 0
    curr_time_sketching = 0.0
    curr_time_obj_perturb = 0.0

    # Adjust epsilon and delta
    delta_DP_adjusted = ((removal_size-1)/removal_size) * delta_DP
    epsilon_adjusted  = ((removal_size-1)/removal_size) * eps

    sigma_DP = 2.0 * np.log(1.25/delta_DP) / (eps**2)
    sigma_matrix = solve_sigma_renyi(sigma_DP, k_opt_second_order, d, delta_DP_adjusted, epsilon_adjusted, 1.0)

    # Train model with privacy
    model_hyperparameters = SmallCNN(output_dim=output_dim, num_classes=2)
    optimizer = optim.Adam(model_hyperparameters.parameters(), lr=lr_full, weight_decay=0.0)
    _, _, _, model_dp = train_with_dp(model_hyperparameters, optimizer, train_loader, test_loader, epochs, lr_full, clip_norm_dpsgd,
                        delta_DP/removal_size, eps/removal_size, device=DEVICE)

    # Extract Features
    train_feats, train_labels = extract_cnn_features(model_dp, train_loader, DEVICE)
    test_feats,  test_labels  = extract_cnn_features(model_dp, test_loader,  DEVICE)

    # Convert to NumPy for scikit‑learn
    X_train = train_feats.numpy()
    y_train = train_labels.numpy()
    X_test  = test_feats.numpy()
    y_test  = test_labels.numpy()

    norm_fact = np.sqrt(np.max(np.sum(X_train**2, 1)))
    X_train = X_train/norm_fact
    X_test = X_test/norm_fact

    print('Data norm is:' + str(np.max(np.sum(X_train**2, 1))))

    # Repeat the experiment `iters` times for averaging
    for _ in tqdm(range(iters)):

        #######################################################################
        # (1) Objective Perturbation
        #######################################################################
        test_acc_obj_perturb, time_obj_perturb = Objective_perturb_LBFGS(X_train, y_train, X_test, y_test,\
                        lambda_param, epsilon_adjusted, delta_DP_adjusted,\
                        num_steps=500,  tol=1e-6, verbose=False,\
                        device=torch.device('cpu'))

        curr_time_obj_perturb += time_obj_perturb
        curr_test_acc_loss_perturb.append(test_acc_obj_perturb)

        #######################################################################
        # (2) Second order approximation
        #######################################################################
        test_acc_second_order, \
          time_second_order, final_theta_cheb = \
                    second_order_cheb(X_train, X_test, y_train, y_test, k_opt_second_order, n, d, tau, sigma_matrix)
        curr_time_sketching += time_second_order
        curr_test_acc.append(test_acc_second_order)


    # End of for _ in tqdm(range(iters))
    opt_acc_ours.append(np.mean(curr_test_acc))
    opt_acc_ours_std.append(t_value * np.std(curr_test_acc)/np.sqrt(iters))
    opt_acc_objective.append(np.mean(curr_test_acc_loss_perturb))
    opt_acc_objective_std.append(t_value * np.std(curr_test_acc_loss_perturb)/np.sqrt(iters))

    if eps_idx == (len(epsilon_values) - 1):
      total_time_ours += curr_time_sketching / iters
      total_time_obj_perturb += curr_time_obj_perturb / iters

    # Print some diagnostics
    print('Test Acc Sketching: ' + str(np.mean(curr_test_acc)))
    print('Test Acc STD Sketching: ' + str( np.std(curr_test_acc)))


    print('Test Acc Objective Pertubration: ' + str(np.mean(curr_test_acc_loss_perturb)))
    print('Test Acc STD Objective Pertubration: ' + str(np.std(curr_test_acc_loss_perturb)))

    print('Total time ours: ' + str(curr_time_sketching / iters))
    print('Total time Objective Perturb: ' + str(curr_time_obj_perturb / iters))

## Plot results

In [ ]:
plt.figure(figsize=(9,8), constrained_layout=True)
# Baseline (non-private) horizontal line
plt.axhline(
    y=1.0 - baseline_test_acc,
    color='black',
    linestyle='--',
    linewidth=4.0,
    label='Baseline (Non-private)'
)
# Store line objects and labels (for legend management)
line_handles = []
line_labels = []

# (A) Plot Ours
line1, = plt.plot(
    epsilon_values,
    [1.0 - element for element in opt_acc_ours],
    marker='o',
    markersize=12,
    color='orangered',
    label=fr"Gaussian mixing (ours) $\frac{{k}}{{d}}$: {k_opt_second_order/X_train.shape[1]:.3f}"
)
# Shading for standard errors
y_error_upper = [
    max(0.0, 1.0 - (m - s/2.0)) for (m, s) in zip(opt_acc_ours, opt_acc_ours_std)
]
y_error_lower = [
    max(0.0, 1.0 - (m + s/2.0)) for (m, s) in zip(opt_acc_ours, opt_acc_ours_std)
]
plt.fill_between(
    epsilon_values, y_error_lower, y_error_upper,
    alpha=0.2, color='orangered'
)
line_handles.append(line1)

# (C) Plot Objective Perturbation (once for i=0)
line3, = plt.plot(
    epsilon_values,
    [1.0 - elem for elem in opt_acc_objective],
    marker='*',
    markersize=12,
    color='blue',
    label='Objective Perturbation (Guo et al., 2020)'
)
y_error_upper = [
    max(0.0, 1.0 - (m - s/2.0)) for (m, s) in zip(opt_acc_objective, opt_acc_objective_std)
]
y_error_lower = [
    max(0.0, 1.0 - (m + s/2.0)) for (m, s) in zip(opt_acc_objective, opt_acc_objective_std)
]
plt.fill_between(
    epsilon_values, y_error_lower, y_error_upper,
    alpha=0.2, color='blue'
)
line_handles.append(line3)

# Axis labels
plt.xlabel(r"$\epsilon_{\text{DP}}$", fontsize=24)
plt.ylabel(r"$Test\ Error\ Rate$", fontsize=24)
plt.grid(True)

# Build a multi-row legend
handles, labels = plt.gca().get_legend_handles_labels()
split1 = len(handles) // 3
split2 = 2 * len(handles) // 3

legend1 = plt.legend(handles[:split1], labels[:split1], loc="lower center",
                         bbox_to_anchor=(0.5, -0.35), ncol=2, frameon=False,
                         fontsize=24, labelspacing=1.0)

legend2 = plt.legend(handles[split1:split2], labels[split1:split2], loc="lower center",
                     bbox_to_anchor=(0.5, -0.45), ncol=2, frameon=False,
                     fontsize=24, labelspacing=1.0)

legend3 = plt.legend(handles[split2:], labels[split2:], loc="lower center",
                     bbox_to_anchor=(0.5, -0.55), ncol=2, frameon=False,
                     fontsize=24, labelspacing=1.0)

# Add the first two legends to the current plot
plt.gca().add_artist(legend1)
plt.gca().add_artist(legend2)
plt.title(dataset_type + ' Classification', fontsize = 28)

line1 = f"Faster by : {total_time_obj_perturb/total_time_ours:.2f}x"

# Combine all lines with the DP delta string
textstr = (
    fr"$\delta_{{\mathrm{{DP}}}} = \min(10^{{-6}}, \frac{{0.01}}{{n}})$" + "\n"
    + f"{line1}\n"
)


plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xscale('log')
plt.text(
    0.95, 0.95, textstr, transform=plt.gca().transAxes, fontsize=28,
    verticalalignment='top', horizontalalignment='right',
    bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white'),
    family='monospace'
)

# Save figures
base_name = f"Logistic_oneshot_" + dataset_type + "_optimal_hyperparam"
plt.savefig(base_name + ".pdf", format="pdf", bbox_inches="tight")
plt.show()